## Vlasov equations

We want to solve Vlosov equations on 1  (in $S^1$).

The equations are: 

\begin{align}
\partial_t f + p \partial_x f &= \frac{q}{m}E \partial_p f \\
\partial_t E = -4\pi J \\
\partial_x E = 4\pi \rho \\
\end{align}

Where $f = f(t,x,p)$ is a distribution function in phase espace. 
Note that the invarian volume element is, 

$$
dP = dp_x/p_0
$$ 

We define, the particle number with respect to an observer with four-velocity $u^a$,

$$
N = \int f(x,p)(-u \cdot p)\; dP
$$ 

Thus, for the observer at rest in the coordinate system $(t,x)$ we get

$$
N = \int f(x,p) p_0 \; dP = \int f(x,p) \; dp_x
$$ 

Otherwise one has the four-vector particle density,

$$
N^a = \int f(x,p) p^a \; dP = \int f(x,p) \frac{p^a}{p_0} \; dp_x
$$

Thus, 

$$
N^x = \int f(x,p) \frac{p_x}{p_0} \; dp_x
$$

Likewise we have the energy-momentum tensor,

$$
T^{ab} = \int p^a p^b f(x,p) dP 
$$

So, 

$$
T^{00} = \int p^0 f(x,p) dp_x = m \int \sqrt{1 + p^2/m^2} dp
$$




\begin{align}
\rho(t,x) &:= q\int f(t,x,p) \; dp - n_0 \\
n_0 &:= q \int \int f(t,x,p) \; dp \; dx / V\\
J(t,x) &:= q \int v f(t,x,p) \; dp, \\
v &:= \frac{\frac{p}{m}}{\sqrt{1 + \frac{p^2}{m^2}}} \\ 
\end{align}

And $E = E(t,x)$ is the electric field.


The equilibrium distribution function is: 

$$
f(\gamma) = \frac{\gamma^2 \beta}{\theta K_2(\frac{1}{\theta})}\; e^{-\frac{\gamma}{\theta}} \;\;\;\; \gamma = \frac{1}{\sqrt{1 - \beta^2}}
$$ 

or

$$
f(p) = \frac{1}{4\pi m^3 c^3 \theta K_2(\frac{1}{\theta})} \; e^{-\frac{\gamma(p)}{\theta}} \;\;\;\;\; \gamma(p) = \sqrt{1 + (\frac{p}{m})^2}
$$

and $K_2$ is the Bessel function of second kind.

In [ ]:
using Plots
using Statistics
using FFTW
FFTW.set_provider!("mkl")
#import Pkg; Pkg.add("FileIO")
using FileIO
using Base.Threads
using Distributions
#Pkg; Pkg.add("DistributedArrays")
println("nthreads = $(nthreads())")

In [ ]:
include("aux_functions_vlasov.jl")

In [ ]:
run_name = "fine_"


landau = false
two_streams = false
landau = true
#two_streams = true
#const Np = 201 # we take even since we need positive and negative values
#const Lx = 1
#const Lp = 0.5 # para cada lado

if landau
    if true #undamped
        const Lx = 39.738 
        const Nx = 3522
        const n = 2
        run_name = run_name * "landau_undamped"
    else #damped 
        const Lx = 7.455
        const Nx = 930
        const n = 15
        run_name = run_name * "landau_damped"
    end

    const Np = 200
    const Lp = 0.5
    
    α = 0.01
    k = 2*π*n/Lx
    θ = 0.001
    run_name = run_name * "$(Nx)_$(Np)_alp2_n$(n)_Th3"
elseif two_streams
    const Nx = 200 # usar par para Fourier transform
    const Lx = 1.0
    const Np = 200
    const Lp = 0.5
    run_name = run_name * "two_streams_"
    k = 2*π*n/Lx
    θ = 0.01
    vel = 0.2
    run_name = run_name * "$(Nx)_$(Np)_vel_p2_v2_Th3"
end

dx = Lx/Nx # periodic boundary conditions
dp = 2*Lp/(Np-1) # Dichlet... or whatever but not periodic

const m = 1
const e =-1
#const q = 1

const κ = 2π/Lx

par_grid = (Nx, dx, Np, dp)

p = [get_p(j,dp,Np)/m for j ∈ 1:Np]
v = [p[j]/sqrt(1+p[j]^2) for j ∈ 1:Np];

run_name


In [ ]:
#E = zeros(Nx)
#ϕ = zeros(Nx)
ρ = zeros(Nx) #charge density
S = zeros(Nx) #carge current
E_K = zeros(Nx) # kinetic energy
#E_E = zeros(Nx) # Electromagnetic energy
P = zeros(Nx) # Momentum
du = zeros(Nx * (Np+1)); # contains f and E


## Initial data

In [ ]:
u = zeros(Nx * (Np+1))

pars = (Nx, dx, Lx, Np, dp, Lp, κ, e)



if landau
    pars_f = (m, θ, α, k)
    u = generate_initial_data!(landau_rel_dist, u, pars_f, pars);
elseif two_streams
    pars_f = (m, θ, 0.2)
    u = generate_initial_data!(counter_streams_rel_dist, u, pars_f, pars);
end



F = reshape(u[1:Nx*Np],(Nx,Np));

plot(F[Nx÷2,:])
#plot_matrix(F)
#save("initial_dist_landau_big.png")

In [ ]:
E_i = deepcopy(u[Nx*Np+1:end])
get_current!(u, S, (Nx, dx, Np, dp, v, m, e));
get_density!(u, ρ, (Nx, dx, Np, dp, m, e))
plot(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))

x = [(i-1)*dx for i in 1:Nx]

plot(layout=(2,2))
plot!(subplot=1,x,ρ, title = "density", legend = :false)
plot!(subplot=2,x,E_i, title = "Electric Field", legend = :false)
plot!(subplot=3,x,S, title = "Current", legend = :false)
plot!(subplot=4,x,E_K, title = "Kinetic Energy", legend = :false)

## Time evolution

In [ ]:
t = 0.0
t_i = 0.0
t_f = 100.0
M = 10001
M_g = 500 + 1 #number of outputs, starting from the initial data
dt = t_f / (M-1)
par_evolv = (t_i, t_f, M, M_g, dt)


In [ ]:
dvx = zeros(Nx)
dvp = zeros(Np)
k1 = zeros(Nx*(Np+1))
k2 = zeros(Nx*(Np+1))
k3 = zeros(Nx*(Np+1))
k4 = zeros(Nx*(Np+1))
p_F = (dx, dp, Nx, Np, v, S, dvx, dvp)
par_RK = (k1, k2, k3, k4)

# total quantities 
Energy_K = zeros(M_g)
Energy_E = zeros(M_g)
EField_T = zeros(M_g)
p_T = zeros(M_g)
n_T = zeros(M_g)
S_T = zeros(M_g)
E_E = 0.0
#T = zeros(M_g)


In [ ]:

j = 1

Energy_K[j]  = sum(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))*dx
Energy_E[j]  = get_E_energy(u,(Nx, dx))
EField_T[j] = sum(u[Nx*Np+1:end])*dx
p_T[j] = sum(get_momentum!(u,P,(Nx, dx, Np, dp)))*dx

get_density!(u, ρ, (Nx, dx, Np, dp, m, e))
get_current!(u, S, (Nx, dx, Np, dp, v, m, e))
n_T[j] = get_total_density!(ρ,(Nx, dx))
S_T[j] = sum(S)/n_T[j]/Nx
#T[1] = var(u[N+1:2N])

### Temporal evolution: This takes a while!

In [ ]:
#include("aux_functions_vlasov.jl")

for k in 2:M
    RK4_Step!(F!,u,t,dt,p_F,par_RK)
    global t = t + dt
    if (k-1) % (M÷(M_g-1)) == 0
        local j = (k-1)÷(M÷(M_g-1))+1
        Energy_K[j]  = sum(get_K_energy!(u,E_K,(Nx, dx, Np, dp)))*dx
        Energy_E[j]  = get_E_energy(u,(Nx, dx))
        EField_T[j] = sum(u[Nx*Np+1:end])*dx
        p_T[j] = sum(get_momentum!(u,P,(Nx, dx, Np, dp)))*dx
        get_density!(u, ρ, (Nx, dx, Np, dp, m, e))
        get_current!(u, S, (Nx, dx, Np, dp, v, m, e))
        n_T[j] = get_total_density!(ρ,(Nx, dx))
        S_T[j] = sum(S)/n_T[j]/Nx
        println("j = $j , t = $t, k = $k, nthreads = $(nthreads())")
    end
  end

In [ ]:
run = Dict("run_name" => run_name, "par_grid" => par_grid, "par_evolv" => par_evolv, "p_Ini" => pars_f, "Energy_E" => Energy_E, "Energy_K" => Energy_K, "E_f" => u[Nx*Np+1:end], "n_F" => ρ, "S_F" => S, "E_T"=> EField_T, "S_T" => S_T, "n_T" => n_T)
save(run_name * "th$(nthreads())_results.jld2", run)

### Shows the final data, warning lot of  memory for big grids! 

In [ ]:
#F = reshape(u[1:Nx*Np],(Nx,Np));
#plot_matrix(F)




In [ ]:
ρ_f = zeros(Nx)
E_f = zeros(Nx)
ϕ_f = zeros(Nx)
S_f = zeros(Nx)


get_density!(u, ρ_f, (Nx, dx, Np, dp, m, e))
n0 = get_total_density!(ρ_f, (Nx, dx))
println("n0 = $(n0)")
get_ϕ!(ϕ_f, ρ_f .- e*n0, κ)
get_E_from_ϕ!(ϕ_f,E_f,dx)


plot(x,E_f,label="from final density", ls=:dash, lw=2)
plot!(x,E_i,label="E_initial")
plot!(x,u[Nx*Np+1:end], label="E_final")
#save("Efield_landau_undamped.png")

In [ ]:
factor = 200
plot(layout=(2,2))
plot!(subplot=1, (Energy_K .- Energy_K[1]), label="Energy_K")
plot!(subplot=1, (Energy_E .- Energy_E[1]), label="Energy_E")
plot!(subplot=2, (Energy_K + Energy_E) ./ (Energy_K[1] + Energy_E[1]) .- 1, label="Total Energy")
plot!(subplot=3, n_T, label="Total density")
plot!(subplot=4, S_T, label="Total Current")
#save("landau_undamped.png")

In [ ]:
plot(ρ_f)

In [ ]:
plot(p_T)

In [ ]:
Energy_K = load(run_name * "th$(nthreads())_results.jld2", "Energy_K")
Energy_E = load(run_name * "th$(nthreads())_results.jld2", "Energy_E")
plot(Energy_K .- Energy_K[1])
plot!(Energy_E)


In [ ]:
println("M = $M, M_g = $M_g")
tt = 0:dt*(M-1)/(M_g-1):t_f

if true #undamped
    plot(tt,Energy_K)
    plot!(tt, 1.0005005 .+ 0.0000196*cos.(0.562*tt .- π/2).^2)
    plot!(tt, 1.0005103 .+ 0.0000196/2*cos.(0.562*2*tt .- π))
else
    plot(tt,log10.(Energy_E))
    plot!(tt, log10.(10^(-7)*1.450*cos.(1.512*tt .- 0).^(2).*exp.(-0.03.* tt)))
end

Los parámetros de fiteo son: 

### Caso undamped

    Lx = 39.738, Nx = 3522, Np = 200, Lp = 0.5

    α = 0.01
    n = 4
    k = 2*π*n/Lx
    θ = 0.001

$E_K = a + b*cos(\omega*t + \alpha)^2$ 

Tenemos $a = 1.0005005$, $b = 0.0000196$ $\omega = 0.562$, $\alpha = -\pi/2$

$E_K = a' + d'*cos(\omega' * t + \alpha')$ 

Tenemos $a'= 1.0005103$, $b'= 0.0000196/2$ $\omega' = 0.562*2$ $\alpha'= - π$

### Caso damped

    Lx = 7.455, Nx = 930, Np = 200, Lp = 0.5

    α = 0.01
    n = 15
    k = 2*π*n/Lx
    θ = 0.001

$E_E = b*cos(\omega*t)^2 e^{-\gamma * t}$ 

Tenemos: $b = 1.450 \; 10^{-7}$, $\omega = 1.512$, $\gamma = 0.03$


#### Note: 

Since we are using a $4\pi$ in the equation for E dot, we need to change E and t to get to the equations in the SHARP paper, for that we have to change time by a factor $\sqrt{4\pi}$. 

Using this factor we get:

#### undamped: #### 

$\omega = \; \sqrt{4π} * 0.562 \;= \;1.9922 $

#### damped: #### 

$\omega = \sqrt{4π} * 1.512 = 5.3599$, $\gamma = \sqrt{4π} * 0.03 = 0.1063$

In [ ]:
sqrt(4π)* 0.03

5.36/4

In [ ]:
using LsqFit

In [ ]:

p0 = [0 ; 1; 0]
@. model(x, p) = p[1]*sin(x*p[2])^2 * exp(-x*p[3]) #*cos(x*p[5] + p[6])

In [ ]:
t_series = [dt*(i-1)*(M-1)/(M_g-1) for i in 1:M_g];
fit = curve_fit(model, t_series, Energy_E, p0);
fit.param